# The goal of this notebook is to create a model that is able to predict the SOC content in the us midwest fields dataset

## importing libraries

In [1]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

## random forest model

In [2]:
dataset = pd.read_csv('data/U.S. Midwest fields/us_midwest_dataset.csv')

labels = np.array(dataset['SOCc'])
features = np.array(dataset.drop('SOCc', axis = 1))

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 2000)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    rf_model = RandomForestRegressor(n_estimators=n_estimators, 
                                     max_depth=max_depth,
                                     min_samples_split=min_samples_split,
                                     min_samples_leaf=min_samples_leaf,
                                     random_state=42)
    
    rf_model.fit(train_features, train_labels)

    score = rf_model.score(test_features, test_labels)
    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

rf = RandomForestRegressor(**study.best_params, random_state=42)
rf.fit(train_features, train_labels)

predictions = rf.predict(test_features)

mae = mean_absolute_error(test_labels, predictions)
r2 = r2_score(test_labels, predictions)

print()
print('mean absolute error :', mae)
print('r2 :', r2)

importances = list(rf.feature_importances_)

feature_list = list(dataset.columns)
feature_list.remove('SOCc')

feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

print()
for pair in feature_importances:
    print('Variable: {:20} Importance: {}'.format(*pair))

[I 2023-11-29 09:44:06,941] A new study created in memory with name: no-name-bf3a7aa9-06ff-4147-8b09-5a85815542b8
[I 2023-11-29 09:44:43,842] Trial 0 finished with value: 0.7997057574457486 and parameters: {'n_estimators': 429, 'max_depth': 27, 'min_samples_split': 6, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.7997057574457486.
[I 2023-11-29 09:46:03,903] Trial 1 finished with value: 0.7966592968109444 and parameters: {'n_estimators': 1028, 'max_depth': 23, 'min_samples_split': 6, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.7997057574457486.
[I 2023-11-29 09:46:38,177] Trial 2 finished with value: 0.7980979132899266 and parameters: {'n_estimators': 433, 'max_depth': 19, 'min_samples_split': 4, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.7997057574457486.
[I 2023-11-29 09:48:07,381] Trial 3 finished with value: 0.802590418523729 and parameters: {'n_estimators': 963, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 5}. Best is trial 3 with valu


mean absolute error : 0.19588535909464264
r2 : 0.8051389251108476

Variable: sample_depth_min     Importance: 0.59
Variable: X                    Importance: 0.07
Variable: BD                   Importance: 0.05
Variable: Y                    Importance: 0.04
Variable: ndvi_mean            Importance: 0.03
Variable: evi2_mean            Importance: 0.03
Variable: sample_depth_max     Importance: 0.02
Variable: evi_mean             Importance: 0.02
Variable: savi_mean            Importance: 0.02
Variable: gndvi_mean           Importance: 0.01
Variable: ndwi_mean            Importance: 0.01
Variable: dem                  Importance: 0.01
Variable: T_GRAVEL             Importance: 0.0
Variable: T_SAND               Importance: 0.0
Variable: T_SILT               Importance: 0.0
Variable: T_CLAY               Importance: 0.0
Variable: T_USDA_TEX_CLASS     Importance: 0.0
Variable: T_REF_BULK_DENSITY   Importance: 0.0
Variable: T_BULK_DENSITY       Importance: 0.0
Variable: T_OC             

## XGBoost model

In [3]:
dataset = pd.read_csv('data/U.S. Midwest fields/us_midwest_dataset.csv')

labels = np.array(dataset['SOCc'])
features = np.array(dataset.drop('SOCc', axis = 1))

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 2000)
    max_depth = trial.suggest_int('max_depth', 2, 32)

    xgboost = XGBRegressor(n_estimators=n_estimators, max_depth=max_depth)
    xgboost.fit(train_features, train_labels)

    score = xgboost.score(test_features, test_labels)
    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

xgboost = XGBRegressor(**study.best_params)
xgboost.fit(train_features, train_labels)

predictions = xgboost.predict(test_features)

mae = mean_absolute_error(test_labels, predictions)
r2 = r2_score(test_labels, predictions)

print()
print('mean absolute error :', mae)
print('r2 :', r2)

importances = list(xgboost.feature_importances_)

feature_list = list(dataset.columns)
feature_list.remove('SOCc')

feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

print()
for pair in feature_importances:
    print('Variable: {:20} Importance: {}'.format(*pair)) 

[I 2023-11-29 11:37:03,548] A new study created in memory with name: no-name-fd308d89-36c2-44b6-9b20-958013ec4ddf


[I 2023-11-29 11:37:05,143] Trial 0 finished with value: 0.7634144183180037 and parameters: {'n_estimators': 1850, 'max_depth': 15}. Best is trial 0 with value: 0.7634144183180037.
[I 2023-11-29 11:37:07,337] Trial 1 finished with value: 0.7672713572299453 and parameters: {'n_estimators': 198, 'max_depth': 22}. Best is trial 1 with value: 0.7672713572299453.
[I 2023-11-29 11:37:11,560] Trial 2 finished with value: 0.7670564464518197 and parameters: {'n_estimators': 1993, 'max_depth': 27}. Best is trial 1 with value: 0.7672713572299453.
[I 2023-11-29 11:37:12,706] Trial 3 finished with value: 0.7634144183180037 and parameters: {'n_estimators': 647, 'max_depth': 15}. Best is trial 1 with value: 0.7672713572299453.
[I 2023-11-29 11:37:17,037] Trial 4 finished with value: 0.7676335503914402 and parameters: {'n_estimators': 630, 'max_depth': 32}. Best is trial 4 with value: 0.7676335503914402.
[I 2023-11-29 11:37:17,751] Trial 5 finished with value: 0.7885687416146565 and parameters: {'n_es


mean absolute error : 0.18758527535597738
r2 : 0.81504402768213

Variable: sample_depth_min     Importance: 0.3700000047683716
Variable: precipipation        Importance: 0.14000000059604645
Variable: Y                    Importance: 0.10000000149011612
Variable: X                    Importance: 0.07000000029802322
Variable: dem                  Importance: 0.07000000029802322
Variable: evi2_mean            Importance: 0.05999999865889549
Variable: savi_mean            Importance: 0.05000000074505806
Variable: ndvi_mean            Importance: 0.029999999329447746
Variable: evi_mean             Importance: 0.029999999329447746
Variable: BD                   Importance: 0.019999999552965164
Variable: gndvi_mean           Importance: 0.019999999552965164
Variable: ndwi_mean            Importance: 0.019999999552965164
Variable: sample_depth_max     Importance: 0.009999999776482582
Variable: temperature          Importance: 0.009999999776482582
Variable: T_GRAVEL             Importance: 0.0